# In The Name Of GOD
# VisionCraft: AI Text-to-Image Studio

VisionCraft is a user-friendly web application that leverages the power of state-of-the-art diffusion models to transform your text descriptions into stunning images. Built with Hugging Face and Gradio, it provides a seamless and interactive creative experience.

### Step 1: Installing Required Libraries

In [ ]:
!pip install diffusers transformers accelerate safetensors invisible_watermark gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.3 MB/s eta 0:00:00


### Step 2: Code Structure with a Class

To create robust and maintainable code, we define a class called `ImageGenerator`. This class will manage everything related to the models:

- **Model Caching**
- **Pipeline Management**
- **Encapsulation**

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
import gradio as gr
from typing import Dict

class ImageGenerator:
    """A class to handle loading diffusion models and generating images."""
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.pipelines: Dict[str, torch.nn.Module] = {}
        self.model_map = {
            "SDXL": {
                "id": "stabilityai/stable-diffusion-xl-base-1.0",
                "pipeline": StableDiffusionXLPipeline,
                "params": {"variant": "fp16"}
            },
            "SD 1.5": {
                "id": "runwayml/stable-diffusion-v1-5",
                "pipeline": StableDiffusionPipeline,
                "params": {}
            }
        }
        print(f"Using device: {self.device}")

    def _load_model(self, model_name: str):
        """Loads a model if it's not already cached."""
        if model_name not in self.pipelines:
            print(f"Loading model: {model_name}")
            model_info = self.model_map[model_name]
            pipeline = model_info["pipeline"].from_pretrained(
                model_info["id"],
                torch_dtype=torch.float16,
                use_safetensors=True,
                **model_info["params"]
            )
            self.pipelines[model_name] = pipeline.to(self.device)
            print(f"Model {model_name} loaded.")

    def generate(self, prompt: str, model_name: str, cfg_scale: float, width: int, height: int):
        """Generates an image based on the selected model and parameters."""
        self._load_model(model_name)
        pipe = self.pipelines[model_name]

        print(f"Generating with {model_name}: prompt='{prompt}', cfg={cfg_scale}, size={width}x{height}")

        image = pipe(
            prompt=prompt,
            guidance_scale=cfg_scale,
            width=int(width),
            height=int(height),
            num_inference_steps=25
        ).images[0]

        return image


### Step 3: Building the Gradio UI

In [ ]:
image_generator = ImageGenerator()

def update_resolution_sliders(model_name):
    if model_name == "SDXL":
        return gr.Slider(value=1024), gr.Slider(value=1024)
    else: 
        return gr.Slider(value=512), gr.Slider(value=512)

with gr.Blocks(theme=gr.themes.Soft(), css="body {background-color: #f4f4f4;}") as demo:
    gr.Markdown(
        """
        # 🎨 Text-to-Image Studio 🎨
        Select a model, write a prompt, and see your imagination come to life!
        """
    )
    with gr.Row():
        with gr.Column(scale=2):
            prompt = gr.Textbox(
                label="✨ Your Creative Prompt",
                placeholder="e.g., A majestic lion wearing a crown, studio lighting, epic, detailed",
                lines=3
            )
            model_selector = gr.Radio(
                ["SDXL", "SD 1.5"],
                label="🤖 Select AI Model",
                value="SDXL"
            )
            with gr.Accordion("⚙️ Advanced Settings", open=False):
                cfg_slider = gr.Slider(minimum=1, maximum=15, value=7.5, label="CFG Scale (Prompt Adherence)")
                with gr.Row():
                    width_slider = gr.Slider(minimum=512, maximum=1024, step=64, value=1024, label="Width")
                    height_slider = gr.Slider(minimum=512, maximum=1024, step=64, value=1024, label="Height")

            btn = gr.Button("Generate Image", variant="primary")

        with gr.Column(scale=1):
            image_output = gr.Image(label="🖼️ Generated Image", show_label=False)

    btn.click(
        fn=image_generator.generate,
        inputs=[prompt, model_selector, cfg_slider, width_slider, height_slider],
        outputs=image_output
    )

    model_selector.change(
        fn=update_resolution_sliders,
        inputs=model_selector,
        outputs=[width_slider, height_slider]
    )

demo.launch(debug=True, share=True)

Using device: cuda
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6cbeba6ba972241d81.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading model: SDXL... This may take a moment.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Model SDXL loaded.
Generating with SDXL: prompt='a cat looking a sunset', cfg=7.5, size=1024x1024


  0%|          | 0/25 [00:00<?, ?it/s]

Generating with SDXL: prompt='A majestic lion wearing a crown, studio lighting, epic, detailed', cfg=7.5, size=1024x1024


  0%|          | 0/25 [00:00<?, ?it/s]

Developed By Eiliya Mohebi For Educational Purposes!